Amplitude estimation is the task to ...

The different variant of QAE implemented in Qiskit are:

* (Canonical) Amplitude Estimation [1],
* Maximum Likelihood Amplitude Estimation [2],
* and Iterative Amplitude Estimation [3]

[1] Brassard

[2] Suzuki

[3] Grinko

In [1]:
from qiskit.algorithms import (
    AmplitudeEstimation, IterativeAmplitudeEstimation, MaximumLikelihoodAmplitudeEstimation,
    EstimationProblem
)

At first, let's begin with the original formulation of QAE by Brassard which is implemented in `AmplitudeEstimation`.
As a simple toy problem that we can verify with pen and paper consider the task of finding the amplitude `p` in the state

$$
    |\psi\rangle = \mathcal{A}|0\rangle = \sqrt{1 - p}|0\rangle + \sqrt{p}|1\rangle.
$$

This represents a Bernoulli distribution on a single qubit with sampling probability $\mathrm{Pr}(X = 1) = p$. It can be
prepared with a Pauli-$Y$ rotation as

$$
    \mathcal{A} = RY(\theta_p),
    \theta_p = 2\arcsin(\sqrt{p})
$$

This circuit is easily prepared in Qiskit.

In [6]:
import numpy
from qiskit.circuit import QuantumCircuit

p = 0.234
theta_p = 2 * numpy.arcsin(numpy.sqrt(p))

a_operator = QuantumCircuit(1)
a_operator.ry(theta_p, 0)

a_operator.draw()

┌────────────┐
q_0: ┤ RY(1.0098) ├
     └────────────┘

As a sanity check, let's verify that this actually prepares the right state. 

In [12]:
from qiskit.quantum_info import Statevector

expected = numpy.sqrt(numpy.array([1 - p, p]))
actual = Statevector(a_operator)

print('expected\n', expected)
print('actual\n', actual.data)
print('equivalent?', actual.equiv(expected))  # check if both statevectors are equivalent

expected
 [0.87521426 0.48373546]
actual
 [0.87521426+0.j 0.48373546+0.j]
equivalent? True


To run the QAE experiment we further need to specify how the good state can be identified. For the Bernoulli example a good state is measured if the (only) qubit is in state $|1\rangle$. This information, along with the $\mathcal{A}$ operator is wrapped inside a problem-class: the `EstimationProblem`. 

In [13]:
bernoulli = EstimationProblem(a_operator, objective_qubits=[0])  # the 0th qubit is the objective, it must be |1>

Note that here we have only used the problem class in a basic manner, there are more settings that can be passed:

* a custom $\mathcal{Q}$ operator
* post-processing on the amplitude $a \in [0, 1]$
* a more elaborate callable used to check if a bitstring is a good state (only supported by `MaximumLikelihoodAmplitudeEstimation` and `IterativeAmplitudeEstimation`)

Now we can already run the algorithm. The canonical formulation only requires to set the number of evaluation qubits $m$, with which the error scales as $\mathcal{O}(2^{-m})$.

In [ ]:
# we will use a shot-based simulation
backend = Aer.get_backend('qasm_simulator')

# define the algorithm instance
ae = AmplitudeEstimation(3, quantum_instance=backend)

# solve the problem
ae_result = ae.estimate(bernoulli)

print('Grid-based estimate:', ae_result.estimation)
print('MLE-based estimate:', ae_result.mle)
print('Target:', p)

Per default we also have access to more interesting data, such as the 95%-confidence interval for the result (here for the MLE result):

In [29]:
print('95% confidence interval', ae_result.confidence_interval)

95% confidence interval (0.22938954476417864, 0.24111601042000727)


To compute another confidence interval we can use the `compute_confidence_interval` method, which for each QAE variant supports different calculation techniques. For maximum-likelihood based estimations one can e.g. compute the confidence interval via a Fisher-information based approach or via the likelihood ratio.

In [36]:
fisher_ci = ae.compute_confidence_interval(ae_result, alpha=0.01, kind='fisher')  # algorithms are not stateful, we need to pass the result in
lr_ci = ae.compute_confidence_interval(ae_result, alpha=0.01, kind='likelihood_ratio')
print('99% Fisher CI:', fisher_ci)
print('99% LR CI:', lr_ci)

99% Fisher CI: (0.22752814190011758, 0.2428473622171791)
99% LR CI: (0.22759622370429994, 0.2430038929828027)


We can re-use the same problem instance and compare the result to other amplitude estimation implementations.

In Maximum-Likelihood Amplitude Estimation we can specify the schedule TODO ...

In [34]:
mlae = MaximumLikelihoodAmplitudeEstimation(3, quantum_instance=backend)
mlae_result = mlae.estimate(bernoulli)
print('Result:', mlae_result.estimation)
print('95%-CI:', mlae_result.confidence_interval)

Result: 0.23407452096481732
95%-CI: (0.23166661541248848, 0.23648242651714615)


In Iterative Amplitude Estimation we can specify TODO...

In [35]:
iae = IterativeAmplitudeEstimation(0.01, 0.05, quantum_instance=backend)
iae_result = iae.estimate(bernoulli)
print('Result:', iae_result.estimation)
print('95%-CI:', iae_result.confidence_interval)

Result: 0.23245497723823655
95%-CI: (0.22919209138840219, 0.23571786308807088)


### Specifying the Grover operator directly

All these implementation use powers of the $\mathcal{Q}$ operator. This operator can be generated automatically based on $\mathcal{A}$ and if the good state is simply defined by a set of objective qubits being in state $|1\rangle$. 

But these circuits can quickly become quite large, especially if we don't have such a simple example as the Bernoulli distribution. As an example let's have a look at the circuits from canonical QAE and the iterative variant.

In [49]:
from qiskit import transpile

basis_gates = ['cx', 'u', 'p']
ae_circuit = ae.construct_circuit(bernoulli)
print(ae_circuit.draw())
print(transpile(ae_circuit, basis_gates=basis_gates, optimization_level=0).count_ops())

            ┌───┐     ┌─────────┐                      ┌──────┐
eval_0: ────┤ H ├─────┤0        ├──────────────────────┤0     ├
            ├───┤     │         │┌─────────┐           │      │
eval_1: ────┤ H ├─────┤         ├┤0        ├───────────┤1 qft ├
            ├───┤     │  c_Q**1 ││         │┌─────────┐│      │
eval_2: ────┤ H ├─────┤         ├┤  c_Q**2 ├┤0        ├┤2     ├
        ┌───┴───┴────┐│         ││         ││  c_Q**4 │└──────┘
   q_0: ┤ RY(1.0098) ├┤1        ├┤1        ├┤1        ├────────
        └────────────┘└─────────┘└─────────┘└─────────┘        
OrderedDict([('cx', 111), ('p', 100), ('u', 63)])


In [48]:
iae_circuit = iae.construct_circuit(bernoulli, 4)
print(iae_circuit.draw())
print(transpile(iae_circuit, basis_gates=basis_gates, optimization_level=0).count_ops())

     ┌────────────┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ RY(1.0098) ├┤ Q ├┤ Q ├┤ Q ├┤ Q ├
     └────────────┘└───┘└───┘└───┘└───┘
OrderedDict([('u', 37), ('p', 4)])


In some cases, however, powers of $\mathcal{Q}$ (or the operator itself) can be optimized. In the Bernoulli-case we can calculate that 

$$
\mathcal{Q}^{k} = RY(2k\theta_p)
$$

To pass this into the QAE algorithms we can specify the circuit for $\mathcal{Q}$ directly.

In [62]:
q_operator = QuantumCircuit(1)
q_operator.ry(2 * theta_p, 0)  # for the normal Q without any power, k=1

# monkey patch the power method to ensure we always get efficient circuits
def power(power):
    q_power = QuantumCircuit(1)
    q_power.ry(2 * power * theta_p, 0)
    return q_power

q_operator.power = power 

In [63]:
bernoulli_efficient = EstimationProblem(a_operator, grover_operator=q_operator, objective_qubits=[0])

In [69]:
ae_circuit_efficient = ae.construct_circuit(bernoulli_efficient)
print('Canonical QAE:', transpile(ae_circuit_efficient, basis_gates=basis_gates, optimization_level=0).count_ops())

iae_circuit_efficient = iae.construct_circuit(bernoulli_efficient)
print('Iterative QAE:', transpile(iae_circuit_efficient, basis_gates=basis_gates, optimization_level=0).count_ops())

Canonical QAE: OrderedDict([('p', 15), ('u', 13), ('cx', 12)])
Iterative QAE: OrderedDict([('u', 1)])


And the results are still the same:

In [73]:
ae_result_efficient = ae.estimate(bernoulli)
print('Canonical QAE:')
print('Grid-based estimate:', ae_result_efficient.estimation)
print('MLE-based estimate:', ae_result_efficient.mle)
print()

iae_result_efficient = iae.estimate(bernoulli_efficient)
print('Iterative QAE:')
print('Result:', iae_result_efficient.estimation)

Canonical QAE:
Grid-based estimate: 0.1464466
MLE-based estimate: 0.23454270771649652

Iterative QAE:
Result: 0.23328207021372252


## Post-processing

The QAE algorithm is formulated to estimate amplitudes of quantum states, therefore the result is always in $[0, 1]$. If we want to use the algorithm to estimate other quantities like e.g. expectation values or function integrations that are outside this range we have to apply some scaling to the unit interval and post-processing back to the original domain.

Let's try to compute the integral of $f(x) = 10 \sin^2(x)$ on the domain $[0, 1]$ on $N = 2^n$ discrete points

$$
    \int_0^1 10 \sin^2(x) dx \approx \frac{1}{N}\sum_{i=0}^{N-1} 10 \sin^2\left(\frac{i}{N}\right) 
    = 10 |\langle \cdot 1 | \psi \rangle |^2 
$$

for 

$$
    |\psi\rangle = \frac{1}{\sqrt{N}} \sum_{i=0}^{N - 1}\left[ \cos\left(\frac{i}{N}\right) |i\rangle |0\rangle + \sin\left(\frac{i}{N}\right) |i\rangle |1\rangle  \right]
$$,
where $|\cdot 1\rangle$ means that we only care about the last qubit being in state 1 and trace out the rest.
The state $|\psi\rangle$ can be prepared by a circuit with an initial layer of Hadamards followed by controlled Pauli-$Y$ rotations. For more detail see [3, 4].

Note that we rescaled the amplitude by a factor of 10. In this example the rescaling consists only of a simple scalar multiplication, however for other applications -- such as finance -- the rescaling can be more complex.

[3]

[4]

In [95]:
n = 5
sine_operator = QuantumCircuit(n + 1)
sine_operator.h(range(n))
sine_operator.ry(1 / 2 ** n, n)
for i in range(n):
    sine_operator.cry(2 * 2 ** i / 2 ** n, i, n)

print(sine_operator.draw())

          ┌───┐                                                                
q_0: ─────┤ H ├───────────■────────────────────────────────────────────────────
          ├───┤           │                                                    
q_1: ─────┤ H ├───────────┼─────────────■──────────────────────────────────────
          ├───┤           │             │                                      
q_2: ─────┤ H ├───────────┼─────────────┼───────────■──────────────────────────
          ├───┤           │             │           │                          
q_3: ─────┤ H ├───────────┼─────────────┼───────────┼───────────■──────────────
          ├───┤           │             │           │           │              
q_4: ─────┤ H ├───────────┼─────────────┼───────────┼───────────┼─────────■────
     ┌────┴───┴────┐┌─────┴──────┐┌─────┴─────┐┌────┴─────┐┌────┴────┐┌───┴───┐
q_5: ┤ RY(0.03125) ├┤ RY(0.0625) ├┤ RY(0.125) ├┤ RY(0.25) ├┤ RY(0.5) ├┤ RY(1) ├
     └─────────────┘└────────────┘└─────

If we now run QAE on this example, our result will miss a factor of 10:

In [96]:
sine = EstimationProblem(sine_operator, objective_qubits=[n]) # the last qubit is the objective that must be |1>

iae = IterativeAmplitudeEstimation(0.01, 0.05, quantum_instance=backend)
result = iae.estimate(sine)
print('Result:', result.estimation)
print('CI:', result.confidence_interval)

Result: 0.2718484400425695
CI: (0.2689401949625418, 0.2747566851225971)


We can either remember the rescaling and apply it manually, but we can also enter this information into the `EstimationProblem` and conveniently keep all information in one place.

In [97]:
sine.post_processing = lambda value: 10 * value

result = iae.estimate(sine)
print('Rescaled result:', result.estimation_processed)  # estimation is always unscaled, while *_processed is scaled
print('Rescaled CI:', result.confidence_interval_processed)

Rescaled result: 2.736050750511325
Rescaled CI: (2.706907326621893, 2.7651941744007575)


In [99]:
from scipy.integrate import quad
reference_value, _ = quad(lambda x: 10 * numpy.sin(x)**2, 0, 1)

In [102]:
print('Reference value:', reference_value)
print('Error:', abs(reference_value - result.estimation_processed))

Reference value: 2.7267564329357956
Error: 0.009294317575529565
